In [69]:
import yfinance as yf
import pandas as pd
import requests
import datetime
import mplfinance as mpf
from newsapi import NewsApiClient

In [142]:
def fetch_stock_data(ticker):
    """Fetches historical stock data for the given ticker from Yahoo Finance API"""
    try:
        today = datetime.date.today().strftime("%Y-%m-%d")
        data = yf.download(ticker, start="1900-01-01", end=today, group_by="ticker")
        if data.empty:
            raise Exception("No data available for the given ticker.")
        else:
            data.columns = data.columns.droplevel(0)
            data.index = data.index.date
            data.columns = ["Open", "High", "Low", "Close", "Volume"]
            return data
    except Exception as e:
        print(f"Error fetching stock data for {ticker}: {str(e)}")
        return None

In [143]:
data = fetch_stock_data("AAPL")

[*********************100%***********************]  1 of 1 completed


In [144]:
date_ranges = {
    "1D": 1, "1W": 7, "1M": 30, "YTD": "YTD", "1Y": 365, "5Y": 1825, "MAX": "MAX"
}

selected_range = "1W"

In [145]:
stored_data = data.to_dict()

In [146]:
data.index = pd.to_datetime(data.index)

# Get today's date as a Pandas Timestamp
today = pd.Timestamp.today()

# Apply date filtering
if selected_range == "MAX":
    filtered_data = data  # No filtering, return all data
elif selected_range == "YTD":
    start_of_year = pd.Timestamp(today.year, 1, 1)  # First day of the year
    filtered_data = data[data.index >= start_of_year]
else:
    days = date_ranges[selected_range]
    filtered_data = data[data.index >= today - pd.Timedelta(days=days)]

In [147]:
filtered_data

,Open,High,Low,Close,Volume
2025-03-07,235.110001,241.369995,234.759995,239.070007,46273600
2025-03-10,235.539993,236.160004,224.220001,227.479996,72071200
2025-03-11,223.809998,225.839996,217.449997,220.839996,76137400
2025-03-12,220.139999,221.750000,214.910004,216.979996,62547500


News API Testing for later

In [68]:
api_key = "cb4c4cd982be40bea3e0ac5635fb1360"

In [70]:
newsapi = NewsApiClient(api_key='cb4c4cd982be40bea3e0ac5635fb1360')

In [151]:
def fetch_news(name):
    """Fetches news articles for the given company name from News API"""
    newsapi = NewsApiClient(api_key='cb4c4cd982be40bea3e0ac5635fb1360')
    try:
        response = newsapi.get_everything(q=name, language="en", sort_by="relevancy")
        if response["status"] == "ok":
            news_articles = response["articles"]
            extracted_data = [
                {"date": article["publishedAt"][:10], "title": article["title"], "url": article["url"]}
                for article in news_articles
            ]
            
            return extracted_data

        else:
            raise Exception("Error fetching news articles")
    except Exception as e:
        print(f"Error fetching news articles: {str(e)}")
        return None


In [152]:
news = fetch_news("Apple Inc.")

In [154]:
pd.DataFrame(news)

,date,title,url
0,2025-03-04,People Are Paying Millions to Dine With Donald...,https://www.wired.com/story/people-paying-mill...
1,2025-02-27,"iPhone 16E review: Eh, it’s alright",https://www.theverge.com/reviews/620155/iphone...
2,2025-02-24,Steve Jobs Would Have Celebrated His 70th Birt...,https://www.macrumors.com/2025/02/24/steve-job...
3,2025-03-09,'Severance' director Ben Stiller compares Lumo...,https://www.businessinsider.com/severance-ben-...
4,2025-02-14,Third Point boosts its stake in these ‘Magnifi...,https://www.marketwatch.com/story/third-point-...
...,...,...,...
95,2025-02-18,"Apple TV+ Unveils Sneak Peek at ""BE@RBRICK,"" T...",http://www.thefutoncritic.com/news/2025/02/18/...
96,2025-03-04,Federal Circuit Affirms Partial Win for Patent...,https://ipwatchdog.com/2025/03/04/federal-circ...
97,2025-03-12,Cosmic Mobile Game Releases - Katamari Damacy ...,https://www.trendhunter.com/trends/katamari-da...
98,2025-03-04,Amazon’s Alexa+ features include agentic AI fo...,https://www.digitalcommerce360.com/2025/03/04/...
